# Spam Filter (Bayesian Model)

## Import Libraries to process and store data

In [6]:
using CSV, DataFrames

Precompiling CSV
  ✓ CSV
  1 dependency successfully precompiled in 14 seconds. 23 already precompiled.


In [7]:
raw_df = CSV.read("emails.csv", DataFrame)

Row,Email No.,the,to,ect,and,for,of,a,you,hou,in,on,is,this,enron,i,be,that,will,have,with,your,at,we,s,are,it,by,com,as,from,gas,or,not,me,deal,if,meter,hpl,please,re,e,any,our,corp,can,d,all,has,was,know,need,an,forwarded,new,t,may,up,j,mmbtu,should,do,am,get,out,see,no,there,price,daren,but,been,company,l,these,let,so,would,m,into,xls,farmer,attached,us,information,they,message,day,time,my,one,what,only,http,th,volume,mail,contract,which,month,⋯
,String15,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,Email 1,0,0,1,0,0,0,2,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,4,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
2,Email 2,8,13,24,6,6,2,102,1,27,18,21,13,0,1,61,4,2,0,0,2,0,12,9,95,4,3,3,3,12,3,1,21,1,12,0,1,0,0,2,15,141,0,21,1,1,39,1,0,0,0,0,14,3,0,91,0,2,8,0,0,0,7,2,1,1,5,0,0,3,0,0,0,35,0,0,7,1,46,0,0,3,1,2,0,0,0,2,2,1,3,0,0,0,15,0,1,0,0,0,⋯
3,Email 3,0,0,1,0,0,0,8,0,0,4,2,0,0,0,8,0,0,0,0,0,0,2,0,2,0,0,0,0,2,0,2,0,0,0,0,0,0,0,0,0,3,0,0,0,0,3,0,0,0,0,0,0,0,0,3,0,0,1,0,0,1,0,0,0,0,2,0,0,0,0,0,0,4,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
4,Email 4,0,5,22,0,5,1,51,2,10,1,5,9,2,0,16,2,0,0,1,1,0,2,1,36,3,1,2,0,2,3,0,10,2,5,2,0,1,0,0,10,79,0,0,0,1,21,0,2,0,0,0,3,2,0,49,0,0,5,0,0,1,9,0,0,1,3,0,0,2,0,1,0,27,0,0,1,0,24,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,4,1,0,0,0,0,⋯
5,Email 5,7,6,17,1,5,2,57,0,9,3,12,2,2,0,30,8,0,0,2,0,0,7,0,19,2,4,2,0,4,1,2,6,0,6,0,0,3,0,1,10,71,0,0,0,1,11,8,0,1,0,0,9,2,0,63,0,0,3,0,1,0,1,1,0,0,9,3,0,1,0,1,0,34,1,0,0,0,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,3,0,4,0,0,⋯
6,Email 6,4,5,1,4,2,3,45,1,0,16,12,8,1,0,52,2,0,0,0,1,0,5,5,56,2,7,1,1,10,0,0,10,0,5,0,1,0,0,0,20,112,0,2,0,0,37,2,2,0,0,0,8,0,1,55,0,1,1,0,0,2,0,0,0,0,1,0,0,0,1,0,0,41,0,0,1,0,25,0,0,0,0,2,0,0,0,0,1,1,6,0,0,0,10,0,0,0,0,0,⋯
7,Email 7,5,3,1,3,2,1,37,0,0,9,4,6,2,0,27,1,0,0,0,0,0,7,1,40,0,0,0,0,11,1,5,2,0,6,1,2,4,1,1,2,56,0,0,0,1,13,0,0,0,0,1,7,0,0,45,0,0,4,0,0,0,0,0,1,2,0,0,0,0,0,0,0,18,0,1,4,0,12,0,0,0,0,0,0,0,0,1,0,0,2,2,0,0,9,0,0,0,0,0,⋯
8,Email 8,0,2,2,3,1,2,21,6,0,2,6,2,0,0,28,1,0,1,0,0,5,1,0,23,0,1,0,0,2,0,0,5,0,2,0,0,0,0,1,9,34,1,6,0,0,17,0,0,0,0,0,7,0,0,18,0,0,1,0,0,2,1,0,0,0,1,0,0,0,0,0,0,14,0,0,1,0,7,0,0,0,0,2,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,⋯
9,Email 9,2,2,3,0,0,1,18,0,0,3,3,2,1,0,15,0,1,0,0,0,0,3,2,6,0,0,0,0,0,0,0,3,0,2,0,0,1,0,0,6,38,0,0,0,0,10,1,0,0,1,1,4,0,0,24,0,0,4,0,0,2,0,0,3,0,2,0,0,0,0,0,0,9,1,1,0,0,3,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,4,0,0,1,0,0,⋯


## Process the data into data matrix for analysis

In [9]:
using Languages
using TextAnalysis 

all_words = names(raw_df)[2:end-1]


all_words_text = join(all_words," ")
document = StringDocument(all_words_text)

prepare!(document, strip_articles)
prepare!(document, strip_pronouns)

vocabulary = split(TextAnalysis.text(document))
clean_words_df = raw_df[!, vocabulary]

data_matrix = Matrix(clean_words_df)'

2975×5172 adjoint(::Matrix{Int64}) with eltype Int64:
 0  13  0   5   6   5   3   2   2   4  …   3  0  18  0  0   2  27   0   7  24
 1  24  1  22  17   1   1   2   3  35      1  1   3  1  1   2  11   1   1   5
 0   6  0   0   1   4   3   3   0   0      2  0   1  0  1   3   2   1   0   1
 0   6  0   5   5   2   2   1   0   1      1  0   6  3  0   0   6   0   2   6
 0   2  0   1   2   3   1   2   1   0      2  0   4  1  0   0   5   0   1   5
 0  27  0  10   9   0   0   0   0  16  …   0  0   2  0  0   0   3   0   0   2
 0  18  4   1   3  16   9   2   3   9      7  0  18  2  0   5  23   1   8  23
 0  21  2   5  12  12   4   6   3   4      3  0  11  2  0   6  18   1  11  13
 1  13  0   9   2   8   6   2   2   1      2  0   4  1  1   1   6   3   7   5
 0   0  0   2   2   1   2   0   1   0      0  0   3  0  0   0   3   1   1   4
 0   1  0   0   0   0   0   0   0   0  …   0  0   1  0  0   0   1   0   0   1
 2  61  8  16  30  52  27  28  15  35     26  1  70  7  1  20  98  10  39  99
 0   4  0 

## Labeling data generated

In [13]:
using MLDataUtils

labels = raw_df.Prediction
(x_train,y_train),(x_test,y_test) = splitobs(shuffleobs((data_matrix, labels)), at = 0.7)

(([1 0 … 12 0; 3 2 … 6 1; … ; 0 0 … 0 0; 0 0 … 0 0], [0, 0, 0, 0, 1, 0, 0, 0, 1, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ([0 5 … 3 2; 1 1 … 1 1; … ; 0 0 … 1 0; 0 0 … 0 0], [0, 0, 0, 0, 1, 1, 0, 1, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 1]))

## Create a data structure for the filter

In [1]:
mutable struct BayesSpamFilter
    words_count_ham::Dict{String, Int64}
    words_count_spam::Dict{String, Int64}
    N_ham::Int64
    N_spam::Int64
    vocabulary::Array{String}
    BayesSpamFilter() = new()
end

In [3]:
function words_count(word_data, vocabulary, labels, spam=0)
    count_dict = Dict{String, Int64}()
    n_emails = size(word_data)[2]
    for (i,word) in enumerate(vocabulary)
        count_dict[word] = sum([word_data[i, j] for j in 1:n_emails if labels[j] == spam])
    end
    return count_dict
end

words_count (generic function with 2 methods)

## Developing function to fit and count 

In [19]:
function fit!(model::BayesSpamFilter, x_train, y_train, voc)
    model.vocabulary = voc
    model.words_count_ham = words_count(x_train, model.vocabulary, y_train, 0)
    model.words_count_spam = words_count(x_train, model.vocabulary, y_train, 1)
    model.N_ham = sum(values(model.words_count_ham))
    model.N_spam = sum(values(model.words_count_spam))
    return
end

fit! (generic function with 1 method)

In [14]:
spam_filter = BayesSpamFilter()
fit!(spam_filter, x_train, y_train, vocabulary)

## Develop Bayesian perspective for spam filter

In [20]:
function word_spam_probability(word,words_count_ham,words_count_spam,N_ham, N_spam, n_vocabulary, α)
    ham_prob = (words_count_ham[word] + α) / (N_ham + α *(n_vocabulary))
    spam_prob = (words_count_spam[word] + α) / (N_spam + α * (n_vocabulary))
    return ham_prob, spam_prob
end

word_spam_probability (generic function with 1 method)

In [21]:
function spam_predict(email, model::BayesSpamFilter, α, tol=100)
    ngrams_email = ngrams(StringDocument(email))
    email_words = keys(ngrams_email)
    n_vocabulary = length(model.vocabulary)
    ham_prior = model.N_ham / (model.N_ham + model.N_spam)
    spam_prior = model.N_spam / (model.N_ham + model.N_spam)

    if length(email_words) > tol
        word_freq = values(ngrams_email)
        sort_idx = sortperm(collect(word_freq), rev=true)
        email_words = collect(email_words)[sort_idx][1:tol]
    end

    email_ham_probability = BigFloat(1)
    email_spam_probability = BigFloat(1)

    for word in intersect(email_words, model.vocabulary)
        word_ham_prob, word_spam_prob = word_spam_probability(word, model.words_count_ham, model.words_count_spam, model.N_ham, model.N_spam, n_vocabulary, α)
        email_ham_probability *= word_ham_prob
        email_spam_probability *= word_spam_prob
    end
    return ham_prior * email_ham_probability, spam_prior * email_spam_probability
end

spam_predict (generic function with 2 methods)

## Generate predictions using the filter

In [22]:
function get_predictions(x_test, y_test, model::BayesSpamFilter, α, tol=200)
    N = length(y_test)
    predictions = Array{Int64,1}(undef, N)
    for i in 1:N
        email = string([repeat(string(word, " "), N) for (word, N) in zip(model.vocabulary, x_test[:, i])]...)
        pham, pspam = spam_predict(email, model, α, tol)
        pred = argmax([pham, pspam]) - 1
        predictions[i] = pred
    end

    predictions
end

get_predictions (generic function with 2 methods)

In [23]:
predictions = get_predictions(x_test,y_test,spam_filter,1)

1552-element Vector{Int64}:
 0
 0
 0
 0
 1
 1
 0
 1
 0
 0
 0
 0
 0
 ⋮
 1
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 1

In [24]:
function spam_filter_accuracy(predictions,actual)
    N = length(predictions)
    correct = sum(predictions .== actual)
    accuracy = correct/N
    return accuracy
end

spam_filter_accuracy (generic function with 1 method)

## Result!

In [25]:
spam_filter_accuracy(predictions,y_test)

0.9536082474226805

In [26]:
sum(raw_df[!, :Prediction])/length(raw_df[!, :Prediction])

0.2900232018561485

In [27]:
function spam_filter_confusion_matrix(y_test, predictions)
    # 2x2 matrix is instantiated with zeros
    confusion_matrix = zeros((2, 2))

    confusion_matrix[1, 1] = sum(isequal(y_test[i], 0) & isequal(predictions[i], 0) for i in 1:length(y_test))
    confusion_matrix[1, 2] = sum(isequal(y_test[i], 1) & isequal(predictions[i], 0) for i in 1:length(y_test))
    confusion_matrix[2, 1] = sum(isequal(y_test[i], 0) & isequal(predictions[i], 1) for i in 1:length(y_test))
    confusion_matrix[2, 2] = sum(isequal(y_test[i], 1) & isequal(predictions[i], 1) for i in 1:length(y_test))

    # Now we convert the confusion matrix into a DataFrame 
    confusion_df = DataFrame(prediction=String[], ham_mail=Int64[], spam_mail=Int64[])
    confusion_df = vcat(confusion_df, DataFrame(prediction="Model predicted Ham", ham_mail=confusion_matrix[1, 1], spam_mail=confusion_matrix[1, 2]))
    confusion_df = vcat(confusion_df, DataFrame(prediction="Model predicted Spam", ham_mail=confusion_matrix[2, 1], spam_mail=confusion_matrix[2, 2]))

    return confusion_df
end

spam_filter_confusion_matrix (generic function with 1 method)

In [28]:
confusion_matrix = spam_filter_confusion_matrix(y_test[:], predictions)

Row,prediction,ham_mail,spam_mail
,String,Float64,Float64
1,Model predicted Ham,1084.0,38.0
2,Model predicted Spam,34.0,396.0


In [29]:
ham_accuracy = confusion_matrix[1, :ham_mail] / (confusion_matrix[1, :ham_mail] + confusion_matrix[2, :ham_mail])


0.9695885509838998

In [30]:
spam_accuracy = confusion_matrix[2, :spam_mail] / (confusion_matrix[1, :spam_mail] + confusion_matrix[2, :spam_mail])

0.9124423963133641